In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/insurance/insurance.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use("fivethirtyeight")
sns.distplot(df['age'])
plt.title("Age Distribution", fontsize=30)

In [ ]:
df['sex'].value_counts()

In [ ]:
plt.figure(figsize=(12, 10))
sns.countplot(df['sex'])

In [ ]:
plt.figure(figsize=(12, 10))
sns.distplot(df['bmi'])

In [ ]:
df['children'].value_counts()

In [ ]:
plt.figure(figsize=(12, 10))
sns.countplot(df['children'])

In [ ]:
df['smoker'].value_counts()

In [ ]:
plt.figure(figsize=(12, 10))
sns.countplot(df['smoker'])

In [ ]:
df['region'].value_counts()

In [ ]:
plt.figure(figsize=(12,10))
sns.countplot(df['region'])

In [ ]:
plt.figure(figsize=(12, 10))
sns.distplot(df['charges'])

In [ ]:
sns.pairplot(df)

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
df.columns

In [ ]:
df

In [ ]:
sex = pd.get_dummies(df['sex'], drop_first=True)


In [ ]:
smoker = pd.get_dummies(df['smoker'], drop_first=True)

In [ ]:
region = pd.get_dummies(df['region'], drop_first=True)

In [ ]:
train = pd.concat([df, sex, smoker, region], axis=1)

In [ ]:
df.drop(['sex', 'smoker', 'region'],axis=1, inplace=True)

In [ ]:
train = pd.concat([df, sex, smoker, region], axis=1)

In [ ]:
train['children'].value_counts()

In [ ]:
train

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
x = train.drop(['charges'], axis=1)
y = train['charges']

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33,
                                                  random_state=0 )



In [ ]:
scale = StandardScaler().fit(x_train)
x_train_std = scale.transform(x_train)
x_test_std = scale.transform(x_test)

In [ ]:
x_train_std

In [ ]:
x_test_std

In [ ]:
lr = LinearRegression().fit(x_train_std , y_train)

y_train_pred = lr.predict(x_train_std)
y_test_pred = lr.predict(x_test)

print(lr.score(x_test_std, y_test))

In [ ]:
lr.score(x_train_std, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [ ]:
find_best_model_using_gridsearchcv(x, y)